In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

URL = r'https://www.premierleague.com/players'

In [26]:
driver = webdriver.Edge()
driver.get(URL)
acceptCookies = driver.find_element(By.ID,'onetrust-accept-btn-handler').click()

lastDocHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) #Giving the website a chance to load
    currDocHeight = driver.execute_script("return document.body.scrollHeight")
    #Terminal Condition
    if currDocHeight == lastDocHeight:
        break
    #Updating the heights
    lastDocHeight = currDocHeight 
    
pageData = driver.page_source   
    
driver.quit()

In [27]:
playerLinks = []


req = requests.get(URL)

if req.status_code == 200: #success
    soup = BeautifulSoup(pageData, 'html.parser')
    #Retrieving Links of Players
    links = soup.find('table').find('tbody').find_all('tr')
    for link in links:
        initLink = link.find('a',class_='player__name')
        finalLink = initLink['href']
        playerLinks.append(finalLink) 

prefix = 'https:'
finalPlayerLinks = [prefix + elem for elem in playerLinks]
print(finalPlayerLinks)

['https://www.premierleague.com/players/19970/Max-Aarons/overview', 'https://www.premierleague.com/players/117754/Zach-Abbott/overview', 'https://www.premierleague.com/players/51536/Terry-Ablade/overview', 'https://www.premierleague.com/players/42894/Tyler-Adams/overview', 'https://www.premierleague.com/players/13549/Tosin-Adarabioyo/overview', 'https://www.premierleague.com/players/72541/Tayo-Adaramola/overview', 'https://www.premierleague.com/players/14685/Elijah-Adebayo/overview', 'https://www.premierleague.com/players/108693/Valintino-Adedokun/overview', 'https://www.premierleague.com/players/126317/Simon-Adingra/overview', 'https://www.premierleague.com/players/114666/Adler-Nascimento/overview', 'https://www.premierleague.com/players/4852/Adrián/overview', 'https://www.premierleague.com/players/128023/Daniel-Adu-Adjei/overview', 'https://www.premierleague.com/players/108628/Nayef-Aguerd/overview', 'https://www.premierleague.com/players/134486/Brandon-Aguilera/overview', 'https://w

In [28]:
#TO-DO: FILTER BY POSITION AND ADD EACH TO A SEPARATE LIST
personalInfo = []
personalInfoColNames = [] 

performanceInfo = []
performanceInfoColNames = [] 

playerData = []

cookiesAccepted = False

driver = webdriver.Edge()

#Adjust the number in finalPlayerLinks to specify the number of players you want Selenium to Scrape
for link in finalPlayerLinks[:30]:
    stats = []  

    driver.get(link)
    
    if not cookiesAccepted:
        try:
            acceptCookies = driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()
            cookiesAccepted = True
        except:
            pass

    time.sleep(2) #Giving time for the page to load all data before scraping

    #Player Name
    try:
        fNameElem = driver.find_element(By.CLASS_NAME, 'player-header__name-first')
        fName = fNameElem.text
    except NoSuchElementException:
        fName = ''

    lName = driver.find_element(By.CLASS_NAME,'player-header__name-last').text
    if fName != '' and lName != '':
        playerName = fName + ' ' + lName
    else:
        playerName = lName
  
    #DOB and Height
    personalInfoDiv = driver.find_element(By.CLASS_NAME, 'player-overview__side-widget')
    
    initPersonalData = personalInfoDiv.text.split('\n')
    if 'Height' not in initPersonalData:
        initPersonalData.extend(['Height',np.nan])
    if 'Club' not in initPersonalData:
        initPersonalData.extend(['Club',None])
    indicesNeeded = [1,2,4,5,6,7,8,9,10,11]
    finalPersonalData = [initPersonalData[i] for i in indicesNeeded]

    
    for data in range(0, len(finalPersonalData)):
        if data % 2 == 0:
            personalInfoColNames.append(finalPersonalData[data])
        else:
            personalInfo.append(finalPersonalData[data])
    
    
    
    #Apps, Goals, Assists, Clean Sheets(In Order of DEF and GK only)
    playerStatsDiv = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//div[@data-widget="player-overview-stats"]')))
    
    performanceData = playerStatsDiv.text.split('\n')
    if 'Clean sheets' not in performanceData:
        performanceData.extend(['Clean sheets',np.nan])
   
    
    for data in range(0, len(performanceData)):
        if data % 2 == 0:
            performanceInfoColNames.append(performanceData[data])
        else:
            performanceInfo.append(performanceData[data])
    #Forming Key,Value Pairs
    personalDataDict  = dict(zip(personalInfoColNames,personalInfo))
    personalDataDict['Name'] = playerName

    performanceDataDict = dict(zip(performanceInfoColNames, performanceInfo))

    finalDict = {**personalDataDict,**performanceDataDict}
    playerData.append(finalDict)

driver.quit()




In [29]:
ORDEREDCOLS = ['Name','Nationality','Club','Position','Date of Birth','Height','Appearances','Goals','Assists','Clean sheets']
df = pd.DataFrame(playerData)
df = df[ORDEREDCOLS]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           30 non-null     object
 1   Nationality    30 non-null     object
 2   Club           23 non-null     object
 3   Position       30 non-null     object
 4   Date of Birth  30 non-null     object
 5   Height         23 non-null     object
 6   Appearances    30 non-null     object
 7   Goals          30 non-null     object
 8   Assists        30 non-null     object
 9   Clean sheets   19 non-null     object
dtypes: object(10)
memory usage: 2.5+ KB


In [30]:
df.head(10)

,Name,Nationality,Club,Position,Date of Birth,Height,Appearances,Goals,Assists,Clean sheets
0,Max Aarons,England,Bournemouth,Defender,04/01/2000 (24),178cm,84,0,4,10
1,Zach Abbott,England,Nottingham Forest,Defender,13/05/2006 (17),NaN,0,0,0,0
2,Terry Ablade,Finland,None,Forward,12/10/2001,NaN,0,0,0,NaN
3,Tyler Adams,United States,Bournemouth,Midfielder,14/02/1999 (25),173cm,24,0,0,NaN
4,Tosin Adarabioyo,England,Fulham,Defender,24/09/1997 (26),196cm,71,2,0,16
5,Tayo Adaramola,Ireland,None,Defender,14/11/2003,177cm,0,0,0,0
6,Elijah Adebayo,England,Luton Town,Forward,07/01/1998 (26),193cm,23,9,0,NaN
7,Valintino Adedokun,Ireland,Brentford,Defender,14/02/2003 (21),NaN,0,0,0,0
8,Simon Adingra,Cote D’Ivoire,Brighton & Hove Albion,Forward,01/01/2002 (22),175cm,20,6,1,NaN
9,Adler Nascimento,Portugal,None,Forward,25/11/2004,NaN,0,0,0,NaN


In [31]:
#Creating a new feature 'Age' derieved from DOB
#TO-DO: TEST THE CODE BELOW
df['DoB'] = df['Date of Birth'].str.slice(0,11)
df['Age'] = df['Date of Birth'].apply(lambda x: x[12:14] if len(x) >= 14 else 2024 - int(x[6:11]) )

df.head(10)

,Name,Nationality,Club,Position,Date of Birth,Height,Appearances,Goals,Assists,Clean sheets,DoB,Age
0,Max Aarons,England,Bournemouth,Defender,04/01/2000 (24),178cm,84,0,4,10,04/01/2000,24
1,Zach Abbott,England,Nottingham Forest,Defender,13/05/2006 (17),NaN,0,0,0,0,13/05/2006,17
2,Terry Ablade,Finland,None,Forward,12/10/2001,NaN,0,0,0,NaN,12/10/2001,23
3,Tyler Adams,United States,Bournemouth,Midfielder,14/02/1999 (25),173cm,24,0,0,NaN,14/02/1999,25
4,Tosin Adarabioyo,England,Fulham,Defender,24/09/1997 (26),196cm,71,2,0,16,24/09/1997,26
5,Tayo Adaramola,Ireland,None,Defender,14/11/2003,177cm,0,0,0,0,14/11/2003,21
6,Elijah Adebayo,England,Luton Town,Forward,07/01/1998 (26),193cm,23,9,0,NaN,07/01/1998,26
7,Valintino Adedokun,Ireland,Brentford,Defender,14/02/2003 (21),NaN,0,0,0,0,14/02/2003,21
8,Simon Adingra,Cote D’Ivoire,Brighton & Hove Albion,Forward,01/01/2002 (22),175cm,20,6,1,NaN,01/01/2002,22
9,Adler Nascimento,Portugal,None,Forward,25/11/2004,NaN,0,0,0,NaN,25/11/2004,20


In [32]:
ORDEREDCOLS = ['Name','Nationality','Club','Position','DoB','Age','Height','Appearances','Goals','Assists','Clean sheets']

initDF = df.drop('Date of Birth',axis=1)[ORDEREDCOLS]

initDF.head(10)

,Name,Nationality,Club,Position,DoB,Age,Height,Appearances,Goals,Assists,Clean sheets
0,Max Aarons,England,Bournemouth,Defender,04/01/2000,24,178cm,84,0,4,10
1,Zach Abbott,England,Nottingham Forest,Defender,13/05/2006,17,NaN,0,0,0,0
2,Terry Ablade,Finland,None,Forward,12/10/2001,23,NaN,0,0,0,NaN
3,Tyler Adams,United States,Bournemouth,Midfielder,14/02/1999,25,173cm,24,0,0,NaN
4,Tosin Adarabioyo,England,Fulham,Defender,24/09/1997,26,196cm,71,2,0,16
5,Tayo Adaramola,Ireland,None,Defender,14/11/2003,21,177cm,0,0,0,0
6,Elijah Adebayo,England,Luton Town,Forward,07/01/1998,26,193cm,23,9,0,NaN
7,Valintino Adedokun,Ireland,Brentford,Defender,14/02/2003,21,NaN,0,0,0,0
8,Simon Adingra,Cote D’Ivoire,Brighton & Hove Albion,Forward,01/01/2002,22,175cm,20,6,1,NaN
9,Adler Nascimento,Portugal,None,Forward,25/11/2004,20,NaN,0,0,0,NaN


In [33]:
initDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          30 non-null     object
 1   Nationality   30 non-null     object
 2   Club          23 non-null     object
 3   Position      30 non-null     object
 4   DoB           30 non-null     object
 5   Age           30 non-null     object
 6   Height        23 non-null     object
 7   Appearances   30 non-null     object
 8   Goals         30 non-null     object
 9   Assists       30 non-null     object
 10  Clean sheets  19 non-null     object
dtypes: object(11)
memory usage: 2.7+ KB


In [34]:
#Preforming Simple Transformations
COLS_TOBE_CASTED = ['Age','Height','Appearances','Goals','Assists','Clean sheets']
COLS_TOBE_FILLED = ['Age','Height','Clean sheets']

initDF['Height'] = initDF['Height'].str.slice(0,3)

initDF[COLS_TOBE_FILLED] =initDF[COLS_TOBE_FILLED].fillna(0)

initDF.to_csv('NOT_CLEAN.csv',index=False)


#Saving the final form to a new dataframe
finalDF = initDF

finalDF[COLS_TOBE_CASTED] = finalDF[COLS_TOBE_CASTED].astype(int)

finalDF.to_csv('CLEAN.csv',index=False)

In [35]:
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          30 non-null     object
 1   Nationality   30 non-null     object
 2   Club          23 non-null     object
 3   Position      30 non-null     object
 4   DoB           30 non-null     object
 5   Age           30 non-null     int32 
 6   Height        30 non-null     int32 
 7   Appearances   30 non-null     int32 
 8   Goals         30 non-null     int32 
 9   Assists       30 non-null     int32 
 10  Clean sheets  30 non-null     int32 
dtypes: int32(6), object(5)
memory usage: 2.0+ KB


In [36]:
finalDF.head(10)

,Name,Nationality,Club,Position,DoB,Age,Height,Appearances,Goals,Assists,Clean sheets
0,Max Aarons,England,Bournemouth,Defender,04/01/2000,24,178,84,0,4,10
1,Zach Abbott,England,Nottingham Forest,Defender,13/05/2006,17,0,0,0,0,0
2,Terry Ablade,Finland,None,Forward,12/10/2001,23,0,0,0,0,0
3,Tyler Adams,United States,Bournemouth,Midfielder,14/02/1999,25,173,24,0,0,0
4,Tosin Adarabioyo,England,Fulham,Defender,24/09/1997,26,196,71,2,0,16
5,Tayo Adaramola,Ireland,None,Defender,14/11/2003,21,177,0,0,0,0
6,Elijah Adebayo,England,Luton Town,Forward,07/01/1998,26,193,23,9,0,0
7,Valintino Adedokun,Ireland,Brentford,Defender,14/02/2003,21,0,0,0,0,0
8,Simon Adingra,Cote D’Ivoire,Brighton & Hove Albion,Forward,01/01/2002,22,175,20,6,1,0
9,Adler Nascimento,Portugal,None,Forward,25/11/2004,20,0,0,0,0,0


In [37]:
#Filtering by positions to drop columns that are unnecessary
CONDITIONS = (finalDF['Position'] == 'Goalkeeper') | (finalDF['Position'] == 'Defender')

DEF_GK = finalDF.loc[CONDITIONS] 

DEF_GK.to_csv('DEF_GK.csv',index=False)

DEF_GK.head(10)

,Name,Nationality,Club,Position,DoB,Age,Height,Appearances,Goals,Assists,Clean sheets
0,Max Aarons,England,Bournemouth,Defender,04/01/2000,24,178,84,0,4,10
1,Zach Abbott,England,Nottingham Forest,Defender,13/05/2006,17,0,0,0,0,0
4,Tosin Adarabioyo,England,Fulham,Defender,24/09/1997,26,196,71,2,0,16
5,Tayo Adaramola,Ireland,None,Defender,14/11/2003,21,177,0,0,0,0
7,Valintino Adedokun,Ireland,Brentford,Defender,14/02/2003,21,0,0,0,0,0
10,Adrián,Spain,Liverpool,Goalkeeper,03/01/1987,37,190,139,0,0,38
12,Nayef Aguerd,Morocco,West Ham United,Defender,30/03/1996,27,190,37,3,0,7
16,Anel Ahmedhodzic,Bosnia & Herzegovina,Sheffield United,Defender,26/03/1999,24,195,21,1,0,1
17,Ola Aina,Nigeria,Nottingham Forest,Defender,08/10/1996,27,184,48,3,2,7
18,Rayan Aït-Nouri,Algeria,Wolverhampton Wanderers,Defender,06/06/2001,22,180,88,3,4,10


In [38]:
#Filtering by positions to drop columns that are unnecessary
CONDITIONS = (finalDF['Position'] == 'Midfielder') | (finalDF['Position'] == 'Forward')

MID_FWD = finalDF.loc[CONDITIONS].drop('Clean sheets',axis=1) 

MID_FWD.to_csv('MID_FWD.csv',index=False)

MID_FWD.head(10)

,Name,Nationality,Club,Position,DoB,Age,Height,Appearances,Goals,Assists
2,Terry Ablade,Finland,None,Forward,12/10/2001,23,0,0,0,0
3,Tyler Adams,United States,Bournemouth,Midfielder,14/02/1999,25,173,24,0,0
6,Elijah Adebayo,England,Luton Town,Forward,07/01/1998,26,193,23,9,0
8,Simon Adingra,Cote D’Ivoire,Brighton & Hove Albion,Forward,01/01/2002,22,175,20,6,1
9,Adler Nascimento,Portugal,None,Forward,25/11/2004,20,0,0,0,0
11,Daniel Adu-Adjei,England,None,Midfielder,21/06/2005,19,177,0,0,0
13,Brandon Aguilera,Costa Rica,None,Midfielder,28/06/2003,21,179,1,0,0
14,Enock Agyei,Belgium,Burnley,Forward,13/01/2005,19,172,0,0,0
15,Naouirou Ahamada,France,Crystal Palace,Midfielder,29/03/2002,21,183,23,0,0
23,Thiago Alcântara,Spain,Liverpool,Midfielder,11/04/1991,32,174,68,2,4
